In [1]:
cd ..

/Users/dorislee/Desktop/Research/covid/covid19-vis


In [2]:
import pandas as pd

In [3]:
%load_ext autoreload
%autoreload 2
from chartlib.covid_chart import CovidChart

In [4]:
df = pd.read_csv("data/quarantine-activity-US-Apr16.csv")
df2 = pd.read_csv("data/reopening-activity-US-Jun9.csv")
df2["Effective Date"] = pd.to_datetime(df2["Effective Date"]).dt.strftime("%m-%d-%Y")
df2.to_csv("data/reopening-activity-US-Jun9.csv")
print(len(df))
print(len(df2))

308
108


In [5]:
combined_df = pd.concat([df,df2])
print (len(combined_df))

416


In [6]:
combined_df.to_csv("data/combined-activity-US-Jun9.csv")

In [7]:
from chartlib import CovidChart, DaysSinceNumReached, days_between
jhu_df = pd.read_csv('./data/jhu-data.csv')
jhu_df = jhu_df[(jhu_df.Country_Region == 'United States') & jhu_df.Province_State.notnull()]

level = 'usa'
qcsv = './data/quarantine-activity-US-Apr16.csv'

days_since = 20
groupcol = 'Province_State'
chart = CovidChart(
    jhu_df,
    groupcol=groupcol,
    start_criterion=DaysSinceNumReached(days_since, 'Confirmed'),
    ycol='Confirmed',
    level=level,
    xcol='Date',
    top_k_groups=30,
    quarantine_df=qcsv  # should have a column with same name as `groupcol`
)

In [8]:
quarantine_df = chart._ingest_usa_quarantine_df("data/quarantine-activity-US-Apr16.csv")
quarantine_df

,Province_State,lockdown_date,lockdown_type,emoji,emoji_string,event_index,Coverage
0,Oregon,02-28-2020,"Gatherings Banned, Gatherings (>250) Banned",🚫👨‍👩‍👧‍👦,g,0,Statewide
118,Rhode Island,03-17-2020,"Gatherings Banned, Gatherings (>25) Banned, Re...",🚫👨‍👩‍👧‍👦,g,0,Statewide
118,Rhode Island,03-17-2020,"Gatherings Banned, Gatherings (>25) Banned, Re...",🚫🍔,r,1,Statewide
119,South Carolina,03-17-2020,"Restaurants; Gatherings Banned, Gatherings (>5...",🚫🍔,r,0,Statewide
119,South Carolina,03-17-2020,"Restaurants; Gatherings Banned, Gatherings (>5...",🚫👨‍👩‍👧‍👦,g,1,Statewide
...,...,...,...,...,...,...,...
188,Washington,03-24-2020,Regional Curfew,🚫🏠,l,0,Regional
191,Colorado,03-25-2020,Regional Stay-at-home Order,🚫🏠,l,0,Regional
192,Florida,03-25-2020,Regional Stay-at-home Order,🚫🏠,l,0,Regional
193,Georgia,03-25-2020,Regional Stay-at-home Order,🚫🏠,l,0,Regional


In [9]:
quarantine_df = chart._ingest_usa_quarantine_df("data/reopening-activity-US-Jun9.csv")
quarantine_df

,Province_State,lockdown_date,lockdown_type,emoji,emoji_string,event_index,Coverage
0,New York,04-15-2020,Face covering required,🚫😷,f,0,Statewide
52,Mississippi,05-07-2020,"Restaurants, Non-essential Businesses",🚫🏬,n,0,Statewide
54,Iowa,05-08-2020,"Restaurants, Non-essential Businesses",🍔,q,0,Statewide
54,Iowa,05-08-2020,"Restaurants, Non-essential Businesses",🚫🏬,n,1,Statewide
55,North Carolina,05-08-2020,"Stay-at-home Order Lifted, Closure of Non-esse...",🏠,h,0,Statewide
...,...,...,...,...,...,...,...
68,Washington,05-11-2020,"Regional Restaurants, Non-essential Businesses",🍔,q,1,Regional
65,Indiana,05-11-2020,"Regional Stay-at-home Order Lifted, Gatherings...",🚫🏬,n,6,Regional
65,Indiana,05-11-2020,"Regional Stay-at-home Order Lifted, Gatherings...",🚫👨‍👩‍👧‍👦,g,7,Regional
65,Indiana,05-11-2020,"Regional Stay-at-home Order Lifted, Gatherings...",🚫👨‍👩‍👧‍👦,g,8,Regional


In [10]:
combined_df = chart._ingest_usa_quarantine_df("data/combined-activity-US-Jun9.csv")
combined_df

,Province_State,lockdown_date,lockdown_type,emoji,emoji_string,event_index,Coverage
0,Oregon,02-28-2020,"Gatherings Banned, Gatherings (>250) Banned",🚫👨‍👩‍👧‍👦,g,0,Statewide
292,Alaska,04-24-2020,"Stay-at-home Order Lifted, Gatherings Banned, ...",🏬,o,0,Statewide
301,Puerto Rico,04-27-2020,Border Closure/Visitor Quarantine,🚫💼,t,0,Statewide
300,Montana,04-27-2020,Closure of Non-essential Businesses,🚫🏬,n,0,Statewide
299,Mississippi,04-27-2020,"Stay-at-home Order Lifted, Closure of Non-esse...",🚫🏬,n,0,Statewide
...,...,...,...,...,...,...,...
224,North Carolina,03-27-2020,Regional Stay-at-home Order,🚫🏠,l,0,Regional
136,Arizona,03-20-2020,"Regional Restaurants,",🚫🍔,r,0,Regional
222,Missouri,03-27-2020,Regional Stay-at-home Order,🚫🏠,l,0,Regional
137,Hawaii,03-20-2020,"Regional Border Closure/Visitor Quarantine, Cu...",🚫🛩,c,1,Regional


In [20]:
combined_df[~combined_df.emoji.str.contains('🚫')]

,Province_State,lockdown_date,lockdown_type,emoji,emoji_string,event_index,Coverage
292,Alaska,04-24-2020,"Stay-at-home Order Lifted, Gatherings Banned, ...",🏬,o,0,Statewide
299,Mississippi,04-27-2020,"Stay-at-home Order Lifted, Closure of Non-esse...",🏠,h,1,Statewide
298,Georgia,04-27-2020,"Restaurants, Non-essential Businesses",🍔,q,1,Statewide
297,Colorado,04-27-2020,Stay-at-home Order Lifted,🏠,h,0,Statewide
292,Alaska,04-24-2020,"Stay-at-home Order Lifted, Gatherings Banned, ...",🍔,q,1,Statewide
...,...,...,...,...,...,...,...
345,Pennsylvania,05-08-2020,"Regional Stay-at-home Order Lifted, Gatherings...",🍔,q,3,Regional
324,Florida,05-04-2020,"Regional Stay-at-home Order Lifted, Restaurant...",🏠,h,0,Regional
324,Florida,05-04-2020,"Regional Stay-at-home Order Lifted, Restaurant...",🍔,q,1,Regional
325,Indiana,05-04-2020,"Regional Stay-at-home Order Lifted, Gatherings...",🏠,h,0,Regional


In [19]:
combined_df[combined_df.emoji.str.contains('🚫')]

,Province_State,lockdown_date,lockdown_type,emoji,emoji_string,event_index,Coverage
0,Oregon,02-28-2020,"Gatherings Banned, Gatherings (>250) Banned",🚫👨‍👩‍👧‍👦,g,0,Statewide
301,Puerto Rico,04-27-2020,Border Closure/Visitor Quarantine,🚫💼,t,0,Statewide
300,Montana,04-27-2020,Closure of Non-essential Businesses,🚫🏬,n,0,Statewide
299,Mississippi,04-27-2020,"Stay-at-home Order Lifted, Closure of Non-esse...",🚫🏬,n,0,Statewide
298,Georgia,04-27-2020,"Restaurants, Non-essential Businesses",🚫🏬,n,0,Statewide
...,...,...,...,...,...,...,...
224,North Carolina,03-27-2020,Regional Stay-at-home Order,🚫🏠,l,0,Regional
136,Arizona,03-20-2020,"Regional Restaurants,",🚫🍔,r,0,Regional
222,Missouri,03-27-2020,Regional Stay-at-home Order,🚫🏠,l,0,Regional
137,Hawaii,03-20-2020,"Regional Border Closure/Visitor Quarantine, Cu...",🚫🛩,c,1,Regional
